<a href="https://colab.research.google.com/github/shawakash/Ml-Projects/blob/Pytorch/02_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Datasets - Toy Datasets

import sklearn
from sklearn.datasets import make_circles

# Samples
n_samples = 1000

# Sets
X, y = make_circles(n_samples,
                    noise=0.03,
                    random_state=42)

len(X), len(y)

In [ ]:
X[:10], y[:10]

In [ ]:
import pandas as pd

circles = pd.DataFrame({"X1": X[:, 0],
                        "X2": X[:, 1],
                        "y": y})

circles.head(10)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x=X[:,0],
            y=X[:,1],
            c=y,
            cmap=plt.cm.RdYlBu)

In [ ]:
# Input and Output shapes

X.shape, y.shape

In [ ]:
X_sample = X[0]
y_sample = y[0]
X_sample, X_sample.shape, y_sample, y_sample.shape

In [ ]:
# Turning Datasets to tensor and train and test splits
import torch

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [ ]:
X.dtype, y.dtype

In [ ]:
# Spliting Dataset Randomly
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
plt.scatter(X_train[:,0],
            X_train[:,1],
            c=y_train)

In [ ]:
# Making Device Agnostic Code
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"

X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [ ]:
# Model

class CircleModelV0(nn.Module):
  def __init__(self):
    super().__init__();

    self.layer_1 = nn.Linear(in_features=2,
                                  out_features=10)

    self.layer_2 = nn.Linear(in_features=10,
                                  out_features=1)

    # self.layer_seq = nn.Sequential(
    #     nn.Linear(in_features=2,
    #               out_features=10),
    #     nn.Linear(in_features=10,
    #               out_features=1)
    # )


  def forward(self, x: torch.tensor) -> torch.tensor:
    # return self._layer_seq(x)
    return self.layer_2(self.layer_1(x))


model_0 = CircleModelV0().to(device)
model_0

In [ ]:
# Using nn.Sequential to make models
torch.manual_seed(42)

model_1 = nn.Sequential(
    nn.Linear(in_features=2,
              out_features=16),
    nn.Linear(in_features=16,
              out_features=1)
).to(device)

model_1

In [ ]:
# Getting some untrained predictions
with torch.inference_mode():
  untrained_preds = model_1(X_test)

torch.round(untrained_preds[:5]), y_test[:5]

In [ ]:
#  Loss function and optimizer

# loss_fn = nn.BCELoss() # BCELoss = no sigmoid built-in
loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.01)


In [ ]:
# Calculate Accuracy - % of correct prediction

def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc

In [ ]:
model_1.eval()

with torch.inference_mode():
  y_logits = model_1(X_test)[:5]

y_logits

In [ ]:
y_preds_probs = torch.sigmoid(y_logits)
y_preds_probs

In [ ]:
torch.round(y_preds_probs)

In [ ]:
y_preds = torch.round(y_preds_probs).squeeze()
y_preds_labels = torch.round(torch.sigmoid(model_1(X_test))[:5]).squeeze()
y_preds_labels == y_preds

In [ ]:
y_preds_labels, y_test[:5]

In [ ]:
# Training the model

torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 100

for epoch in range(epochs):
  model_1.train()

  y_logits = model_1(X_train).squeeze()
  y_pred = torch.round(torch.sigmoid(y_logits))

  loss = loss_fn(y_logits, y_train) #   BCELosswithLogits require y_logits as y_preds
  # But BCELoss requires loss_fn(torch.sigmoid(y_logits), y_test)
  acc = accuracy_fn(y_true=y_train,
                    y_pred=y_pred)

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  # Testing
  model_1.eval()

  with torch.inference_mode():
    test_logits = model_1(X_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))
    test_loss = loss_fn(test_logits, y_test)
    test_acc = accuracy_fn(y_true=y_test,
                           y_pred=test_pred)

  #  Prints
  if epoch%10 == 0:
    print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")



In [ ]:
import requests
from pathlib import Path

if Path("helper_functions.py").is_file():
  print("Helper function already exist :)")
else:
  print("Downloading the helper function file for you:)")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

from helper_functions import plot_predictions, plot_decision_boundary

In [ ]:
# Plot decision boundary
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("Train")
plot_decision_boundary(model_1, X_train, y_train)
plt.subplot(1,2,2)
plt.title("Test")
plot_decision_boundary(model_1, X_test, y_test)

In [ ]:
import torch
from torch import nn

class CircleModelV1(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer_1 = nn.Linear(2,10)
    self.layer_2 = nn.Linear(10, 10)
    self.layer_3 = nn.Linear(10, 1)

  def forward(self, x: torch.Tensor) -> torch.tensor:
    return self.layer_3(self.layer_2(self.layer_1(x)))

model_2 = CircleModelV1().to(device)
model_2

In [ ]:
# Loss function

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(),
                            lr=0.1)

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 1000

for epoch in range(epochs):
  model_2.train()
  y_logits = model_2(X_train).squeeze()
  y_preds = torch.round(torch.sigmoid(y_logits))

  loss = loss_fn(y_logits,
                 y_train)
  acc = accuracy_fn(y_true=y_train,
                    y_pred=y_pred)

  optimizer.zero_grad()

  loss.backward()
  optimizer.step()

  # Testing
  model_2.eval()
  with torch.inference_mode():
    test_logits = model_2(X_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))

    test_loss = loss_fn(test_logits, y_test)
    test_acc = accuracy_fn(y_true=y_test, y_pred=test_pred)

  if epoch%20 == 0:
    print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")



In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("Train")
plot_decision_boundary(model_2, X_train, y_train)
plt.subplot(1, 2, 2)
plt.title("Test")
plot_decision_boundary(model_2, X_test, y_test)

In [ ]:
# Non-Linearity

import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

n_samples = 1000
X, y = make_circles(n_samples,
                    noise=.03,
                    random_state=42)

plt.scatter(X[:,0], X[:, 1], c=y)


In [ ]:
import torch
from sklearn.model_selection import train_test_split

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

X_train, X_test, y_train, y_test = train_test_split(X,y)

device = "cuda" if torch.cuda.is_available() else "cpu"
X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)

In [ ]:
X_train[:5], y_train[:5]

In [ ]:
plt.title("Training Sets")
plt.scatter(X_train[:,0], X_train[:,1], c=y_train)


In [ ]:
plt.title("Testing Sets")
plt.scatter(X_test[:,0], X_test[:,1], c=y_test)

In [ ]:
class CircleModelV2(nn.Module):
  def __init__(self):
    super(CircleModelV2, self).__init__()

    self.layer_1 = nn.Linear(2, 128)
    self.layer_2 = nn.Linear(128, 256)
    self.layer_3 = nn.Linear(256, 128)
    self.layer_4 = nn.Linear(128, 1)
    self.relu = nn.ReLU()

  def forward(self, x: torch.tensor) -> torch.tensor:
    return self.layer_4(
            self.relu(
              self.layer_3(
                self.relu(
                  self.layer_2(
                    self.relu(
                      self.layer_1(x)))))))

model_3 = CircleModelV2.to(device)
model_3

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_3.parameters(),
                            lr=0.01)

In [ ]:
# Training

torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 1000

for epoch in range(epochs):
  model_3.train()

  y_logits = model_3(X_train).squeeze()
  y_pred = torch.round(torch.sigmoid(y_logits))

  loss = loss_fn(y_logits, y_train)
  acc = accuracy_fn(y_true=y_train,
                    y_pred=y_pred)

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  # Testing
  model_3.eval()
  with torch.inference_mode():
    test_logits = model_3(X_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))
    test_loss = loss_fn(test_logits, y_test)
    test_acc = accuracy_fn(y_true=y_test,
                           y_pred=test_pred)

  if epoch%20 == 0:
    print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")
